# Reliability-Based Design Optimization

---

One of the primary uses of uncertainty propagation is to carry out *reliability-based design optimization* (RBDO). 

Reliability-based design optimization is the optimization of some design cost $C$ while constraining the failure probability of some failure modes $\mathbb{P}[\text{Failure}_i] \leq \mathcal{F}_i$.

As a demonstration, we will carry out RBDO on the cantilever beam problem. This leads to the optimization problem:

$$\min_{w, t}\, C_{\text{area}}$$

$$\text{s.t.}\, \mathbb{P}[F_{\text{stress}}], \mathbb{P}[F_{\text{disp}}] \leq \mathcal{F}$$

In this notebook, we will study how to solve this class of problem using grama to approximate failure probabilities, and scipy to carry out multivariate optimization.

In [1]:
import grama as gr
from grama.models import make_cantilever_beam
from scipy.optimize import minimize

X = gr.Intention()
md = make_cantilever_beam()
md.printpretty()

model: Cantilever Beam

  inputs:
    var_det:
      t: [2, 4]
      w: [2, 4]
    var_rand:
      H: (+1) norm, {'loc': 500.0, 'scale': 100.0}
      V: (+1) norm, {'loc': 1000.0, 'scale': 100.0}
      E: (+0) norm, {'loc': 29000000.0, 'scale': 1450000.0}
      Y: (-1) norm, {'loc': 40000.0, 'scale': 2000.0}
    copula:
        Independence copula
  functions:
    cross-sectional area: ['w', 't'] -> ['c_area']
    limit state: stress: ['w', 't', 'H', 'V', 'E', 'Y'] -> ['g_stress']
    limit state: displacement: ['w', 't', 'H', 'V', 'E', 'Y'] -> ['g_disp']


### First Order Reliability Method (FORM)

FORM is a means to approximate *reliability*; the probability $R = 1 - \mathbb{P}[\text{Failure}]$. We give a very brief non-theoretic introduction to FORM below. The *performance measure approach* (PMA) formulation of FORM is given below:

$$\min_{Z}\, g(Z)$$

$$\text{s.t.}\, \|Z\|_2 = \beta$$

where $Z$ are the model random variables transformed to standard normal space. Grama takes care of these transforms automatically so you can focus on building an appropriate model, rather than selecting computationally convenient distributions. The $\beta$ quantity is a measure of reliability, given as $\beta = \Phi^{-1}(\mathcal{R})$ where $\mathcal{R}$ is the target reliability, and $\Phi^{-1}$ is the inverse normal cdf. For instance, $\beta = 3$ corresponds to $\mathcal{R} \approx 1 - 0.00135$.

The result of the PMA optimization $g^*$ is used to replace the probability constraint with $g^* \geq 0$. **The practical effect of FORM is to approximate a high-dimensional integral with an optimization.** This leads to a decrease in accuracy, but a considerable speedup in computation.

An example of evaluating FORM PMA once (for two limit states) is given below. In practice, we will use this routine as a constraint while optimizing the structural cost.

In [6]:
md >> gr.ev_form_pma(df_det="nom", betas=dict(g_stress=3, g_disp=3))

,t,w,H,V,E,Y,g_stress,g_disp
0,3.0,3.0,1.732051,1.732051,0.000000,-1.732051,-0.112386,NaN
0,3.0,3.0,1.143468,2.129280,-1.776579,0.049738,NaN,-0.287475


Note that this design has negative `g_stress, g_disp`; therefore the design studied does not meet the reliability constraints. We will search for a design which does using RBDO.

## Optimization Setup

---

Applying FORM PMA to transform the reliability constraints, we arrive at

$$\min_{w, t}\, C_{\text{area}}(w, t)$$

$$\text{s.t.}\, g_{\text{stress}}^*(w, t), g_{\text{disp}}^*(w, t) \geq 0$$

The following code implements $C_{\text{area}}, g_{\text{stress}}^*, g_{\text{disp}}^*$ as functions of $w, t$, so as to pass them to `scipy.optimize.minimize`.

In [3]:
beta_target = 3

def cost(x):
    w, t = x
    df_det = gr.df_make(w=w, t=t)
    df_res = (
        md
        >> gr.ev_monte_carlo(df_det=df_det, n=1)
    )
    return df_res["c_area"]

def g_stress(x):
    w, t = x
    df_det = gr.df_make(w=w, t=t)
    df_res = (
        md
        >> gr.ev_form_pma(
            df_det=df_det, 
            betas=dict(g_stress=beta_target)
        )
    )
    return df_res["g_stress"]
    
def g_disp(x):
    w, t = x
    df_det = gr.df_make(w=w, t=t)
    df_res = (
        md
        >> gr.ev_form_pma(
            df_det=df_det, 
            betas=dict(g_disp=beta_target)
        )
    )
    return df_res["g_disp"]


## Optimization Execution

---

Below we pass the objective and constraints to the scipy minimization routine; this will carry out RBDO.

In [4]:
## Initial guess
x0 = [3., 3.]
## Optimize
res = minimize(
    cost,
    x0,
    method="SLSQP",
    constraints=[
        {"type": "ineq", "fun": g_stress},
        {"type": "ineq", "fun": g_disp}
    ],
    bounds=[
        (2, 4),
        (2, 4)
    ],
    options={"disp": True}
)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 9.488911449804434
            Iterations: 7
            Function evaluations: 29
            Gradient evaluations: 7


In [8]:
df_opt = gr.df_make(w=res.x[0], t=res.x[1], c_area=res.fun)
df_opt

,w,t,c_area
0,2.459665,3.857806,9.488911


We can check the optimization results with simple Monte Carlo; the failure probabilities for the optimized design should be below the desired `0.00135` level. We must select a sample size roughly $10\times$ the failure probability we are trying to estimate, in order to get a sufficiently accurate estimation.

In [19]:
df_mc = (
    md
    >> gr.ev_monte_carlo(df_det=df_opt[["w", "t"]], n=10 / 0.00135)
)

eval_monte_carlo() is rounding n...


In [20]:
(
    df_mc
    >> gr.tf_mutate(
        fail_stress=(X.g_stress <= 0) * 1,
        fail_disp=(X.g_disp <= 0) * 1
    )
    >> gr.tf_summarize(
        pof_stress=gr.mean(X.fail_stress),
        pof_disp=gr.mean(X.fail_disp),
    )
)

,pof_stress,pof_disp
0,0.00135,0.001215


The design seems to satisfy both constraints.